# Human-AI Information Diffusion Simulation with Real LLM Agents

## Overview

This notebook implements an advanced information diffusion model using actual Large Language Model (LLM) agents instead of pseudo-random implementations. The simulation models how information spreads through a network of AI agents with distinct personalities, roles, and knowledge bases.

### Key Features

- **Real LLM Integration**: Uses OpenAI GPT or Anthropic Claude APIs for authentic agent responses
- **Network Dynamics**: Agents form connections and influence each other based on interactions
- **Advanced Analytics**: Sentiment analysis, engagement scoring, and network centrality metrics
- **Visualization**: Interactive network graphs and time-series analysis
- **Data Integration**: Framework for incorporating real social media data

### Requirements

- Python 3.8+
- API Key for OpenAI or Anthropic
- Required packages: openai/anthropic, pandas, numpy, matplotlib, seaborn, networkx

## 1. Setup and Imports

First, let's import all necessary libraries and set up our environment. Make sure you have set your API keys as environment variables:

```bash
export OPENAI_API_KEY="your-openai-api-key"
# or
export ANTHROPIC_API_KEY="your-anthropic-api-key"
```

In [ ]:
# Core imports
import os
import json
import time
import random
import asyncio
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, field
from datetime import datetime

# Data processing and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, clear_output
import networkx as nx

# API Options (uncomment your choice)
from openai import OpenAI
# from anthropic import Anthropic

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Verify API key is set
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  WARNING: OPENAI_API_KEY not found in environment variables")
    print("Please set: export OPENAI_API_KEY='your-key-here'")
else:
    print("✅ API key detected")

## 2. Configuration

Let's define our simulation configuration parameters. These control API usage, simulation behavior, and visualization settings.

In [ ]:
class Config:
    """Configuration for the simulation"""
    # API Configuration
    API_PROVIDER = "openai"  # or "anthropic"
    OPENAI_MODEL = "gpt-3.5-turbo"
    ANTHROPIC_MODEL = "claude-3-haiku-20240307"
    
    # Simulation Parameters
    MAX_AGENTS = 10
    MAX_TIME_STEPS = 20
    MESSAGE_WINDOW = 5
    MEMORY_LIMIT = 100
    
    # Rate Limiting
    API_DELAY = 1.0  # seconds between API calls
    MAX_TOKENS = 100
    
    # Visualization
    FIGURE_SIZE = (12, 8)
    NETWORK_LAYOUT = "spring"

print(f"Configuration loaded:")
print(f"- API Provider: {Config.API_PROVIDER}")
print(f"- Model: {Config.OPENAI_MODEL if Config.API_PROVIDER == 'openai' else Config.ANTHROPIC_MODEL}")
print(f"- Max agents: {Config.MAX_AGENTS}")
print(f"- API delay: {Config.API_DELAY}s")

## 3. Core Data Structures

### 3.1 Message Class

The `Message` class represents individual communications between agents, including metadata like sentiment and engagement scores.

In [ ]:
@dataclass
class Message:
    """Represents a message in the simulation"""
    sender: str
    content: str
    timestamp: datetime
    topic: str
    sentiment: float = 0.0
    engagement_score: float = 0.0
    recipients: List[str] = field(default_factory=list)
    message_id: str = field(default_factory=lambda: str(random.randint(1000, 9999)))
    
    def __repr__(self):
        return f"Message(from={self.sender}, sentiment={self.sentiment:.2f}, engagement={self.engagement_score:.2f})"

# Example message
example_msg = Message(
    sender="Dr. Chen",
    content="The new fiscal policy requires careful analysis.",
    timestamp=datetime.now(),
    topic="fiscal policy",
    sentiment=0.2,
    engagement_score=0.7
)
print("Example message:", example_msg)

### 3.2 Agent Class

The `Agent` class represents individual AI agents with personalities, expertise, and the ability to generate contextual responses using LLMs.

In [ ]:
@dataclass
class Agent:
    """Enhanced agent with LLM capabilities"""
    name: str
    role: str
    personality: str
    knowledge_base: List[str] = field(default_factory=list)
    memory: List[Message] = field(default_factory=list)
    influence_score: float = 1.0
    connections: List[str] = field(default_factory=list)
    response_history: Dict[str, int] = field(default_factory=dict)
    
    def __post_init__(self):
        """Initialize the API client"""
        if Config.API_PROVIDER == "openai":
            self.api_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        else:
            # self.api_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
            pass
    
    def generate_response(self, context: Dict[str, any], topic: str) -> str:
        """Generate response using actual LLM API with rate limiting"""
        time.sleep(Config.API_DELAY)  # Rate limiting
        
        recent_messages = self._get_recent_messages(limit=5)
        
        system_prompt = self._build_system_prompt(topic)
        user_prompt = self._build_user_prompt(context, recent_messages)
        
        try:
            if Config.API_PROVIDER == "openai":
                response = self.api_client.chat.completions.create(
                    model=Config.OPENAI_MODEL,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt}
                    ],
                    max_tokens=Config.MAX_TOKENS,
                    temperature=0.8
                )
                return response.choices[0].message.content.strip()
            else:
                # Anthropic implementation
                # response = self.api_client.messages.create(...)
                pass
                
        except Exception as e:
            print(f"API Error for {self.name}: {e}")
            return self._generate_fallback_response(topic)
    
    def _build_system_prompt(self, topic: str) -> str:
        """Build system prompt for LLM"""
        return f"""You are {self.name}, a {self.role} with the following personality: {self.personality}.
        You are participating in a discussion about: {topic}
        
        Your areas of expertise: {', '.join(self.knowledge_base[:3])}
        Your connections: {', '.join(self.connections[:3])}
        
        Guidelines:
        - Respond naturally and in character
        - Keep responses concise (1-2 sentences)
        - Reference other participants when relevant
        - Express opinions consistent with your role and personality"""
    
    def _build_user_prompt(self, context: Dict[str, any], recent_messages: List[Message]) -> str:
        """Build user prompt for LLM"""
        conversation = "\n".join([
            f"{msg.sender}: {msg.content}" 
            for msg in recent_messages
        ])
        
        return f"""Recent conversation:
        {conversation}
        
        Current context: {context.get('current_event', 'General discussion')}
        Time step: {context.get('time_step', 0)}
        Trending topics: {context.get('trending_topics', [])}
        
        How do you respond?"""
    
    def _generate_fallback_response(self, topic: str) -> str:
        """Fallback response generation"""
        templates = [
            f"As a {self.role}, I believe {topic} deserves careful consideration.",
            f"From my experience as a {self.role}, {topic} is quite significant.",
            f"I've been analyzing {topic}, and there are important implications.",
        ]
        return random.choice(templates)
    
    def _get_recent_messages(self, limit: int = 5) -> List[Message]:
        """Get recent messages from memory"""
        return self.memory[-limit:] if self.memory else []
    
    def update_connections(self, other_agents: List[str]):
        """Update agent connections based on interactions"""
        for agent_name in other_agents:
            if agent_name != self.name and agent_name not in self.connections:
                if random.random() < 0.3:  # 30% chance to form connection
                    self.connections.append(agent_name)

# Test agent creation
test_agent = Agent(
    name="Test Agent",
    role="Analyst",
    personality="Analytical and thorough",
    knowledge_base=["economics", "policy"],
    influence_score=1.2
)
print(f"Created agent: {test_agent.name} ({test_agent.role})")

## 4. Simulation Engine

The main simulation class that orchestrates agent interactions, tracks metrics, and manages the information diffusion process.

In [ ]:
class InformationDiffusionSimulation:
    """Enhanced simulation with visualization and analytics"""
    
    def __init__(self, agents: List[Agent], topics: List[str]):
        self.agents = {agent.name: agent for agent in agents}
        self.topics = topics
        self.messages: List[Message] = []
        self.time_step = 0
        self.interaction_network = nx.DiGraph()
        self.metrics_history = {
            "reach": [],
            "engagement": [],
            "sentiment": [],
            "message_count": [],
            "active_agents": []
        }
        
        # Initialize network
        for agent in agents:
            self.interaction_network.add_node(agent.name, 
                                            role=agent.role,
                                            influence=agent.influence_score)
    
    def seed_information(self, topic: str, seed_agents: List[str], urgency: str = "normal"):
        """Seed initial information with urgency levels"""
        initial_context = {
            "current_event": f"Breaking: New information about {topic}",
            "urgency": urgency,
            "time_step": 0,
            "trending_topics": [topic]
        }
        
        for agent_name in seed_agents:
            if agent_name in self.agents:
                agent = self.agents[agent_name]
                response = agent.generate_response(initial_context, topic)
                
                message = Message(
                    sender=agent.name,
                    content=response,
                    timestamp=datetime.now(),
                    topic=topic,
                    sentiment=self._analyze_sentiment_advanced(response),
                    engagement_score=random.uniform(0.7, 1.0) if urgency == "high" else random.uniform(0.4, 0.7),
                    recipients=["all"]
                )
                
                self.messages.append(message)
                agent.memory.append(message)
                
                # Update network
                self.interaction_network.nodes[agent.name]['messages_sent'] = 1
                
                print(f"🌱 [SEED] {agent.name}: {response}")
    
    def simulate_time_step(self):
        """Enhanced time step simulation with network effects"""
        self.time_step += 1
        new_messages = []
        active_agents = set()
        
        recent_messages = self._get_recent_public_messages()
        trending_topics = self._calculate_trending_topics()
        
        if not recent_messages:
            print(f"⏸️  No recent messages at time step {self.time_step}")
            return
        
        # Simulate each agent's response
        for agent_name, agent in self.agents.items():
            for message in recent_messages:
                if message.sender == agent_name:
                    continue
                
                # Enhanced response probability calculation
                response_prob = self._calculate_response_probability(agent, message)
                
                if random.random() < response_prob:
                    context = {
                        "current_event": f"Responding to {message.sender}'s message about {message.topic}",
                        "message_sentiment": message.sentiment,
                        "time_step": self.time_step,
                        "trending_topics": trending_topics,
                        "network_influence": self._calculate_network_influence(agent_name)
                    }
                    
                    response = agent.generate_response(context, message.topic)
                    
                    new_message = Message(
                        sender=agent.name,
                        content=response,
                        timestamp=datetime.now(),
                        topic=message.topic,
                        sentiment=self._analyze_sentiment_advanced(response),
                        engagement_score=self._calculate_engagement(response, message),
                        recipients=[message.sender, "all"]
                    )
                    
                    new_messages.append(new_message)
                    agent.memory.append(new_message)
                    active_agents.add(agent_name)
                    
                    # Update interaction network
                    self._update_network(agent_name, message.sender, new_message)
                    
                    print(f"💬 [T{self.time_step}] {agent.name} → {message.sender}: {response}")
        
        # Update simulation state
        self.messages.extend(new_messages)
        self._update_metrics(active_agents)
        
        # Update agent connections
        for agent_name in active_agents:
            self.agents[agent_name].update_connections(list(active_agents))
    
    def _calculate_response_probability(self, agent: Agent, message: Message) -> float:
        """Enhanced probability calculation with network effects"""
        base_prob = 0.2
        
        # Knowledge relevance
        relevance_score = sum(1 for keyword in agent.knowledge_base 
                             if keyword.lower() in message.content.lower()) * 0.15
        
        # Connection influence
        if message.sender in agent.connections:
            base_prob += 0.2
        
        # Engagement influence
        base_prob += message.engagement_score * 0.2
        
        # Agent influence factor
        base_prob *= agent.influence_score
        
        # Sentiment alignment
        if hasattr(agent, 'avg_sentiment'):
            sentiment_diff = abs(agent.avg_sentiment - message.sentiment)
            base_prob *= (1 - sentiment_diff * 0.3)
        
        return min(base_prob + relevance_score, 0.9)
    
    def _analyze_sentiment_advanced(self, text: str) -> float:
        """Advanced sentiment analysis"""
        positive_words = {
            "excellent": 0.9, "great": 0.7, "good": 0.5, "support": 0.6, 
            "agree": 0.5, "important": 0.4, "beneficial": 0.7, "progress": 0.6
        }
        negative_words = {
            "terrible": -0.9, "bad": -0.7, "wrong": -0.6, "disagree": -0.5,
            "concern": -0.4, "worry": -0.5, "problem": -0.6, "crisis": -0.8
        }
        
        text_lower = text.lower()
        sentiment_score = 0.0
        word_count = 0
        
        for word, score in positive_words.items():
            if word in text_lower:
                sentiment_score += score
                word_count += 1
        
        for word, score in negative_words.items():
            if word in text_lower:
                sentiment_score += score
                word_count += 1
        
        return sentiment_score / max(word_count, 1)
    
    def _calculate_trending_topics(self) -> List[str]:
        """Identify trending topics from recent messages"""
        if not self.messages:
            return []
        
        recent_messages = self.messages[-20:]  # Last 20 messages
        topic_counts = {}
        
        for msg in recent_messages:
            topic_counts[msg.topic] = topic_counts.get(msg.topic, 0) + 1
        
        # Sort by frequency
        sorted_topics = sorted(topic_counts.items(), key=lambda x: x[1], reverse=True)
        return [topic for topic, _ in sorted_topics[:3]]
    
    def _calculate_network_influence(self, agent_name: str) -> float:
        """Calculate agent's influence based on network position"""
        if agent_name not in self.interaction_network:
            return 0.0
        
        try:
            centrality = nx.degree_centrality(self.interaction_network)[agent_name]
            return centrality
        except:
            return 0.0
    
    def _update_network(self, sender: str, receiver: str, message: Message):
        """Update interaction network"""
        if self.interaction_network.has_edge(sender, receiver):
            self.interaction_network[sender][receiver]['weight'] += 1
            self.interaction_network[sender][receiver]['messages'].append(message.message_id)
        else:
            self.interaction_network.add_edge(sender, receiver, 
                                            weight=1, 
                                            messages=[message.message_id])
    
    def _calculate_engagement(self, response: str, original_message: Message) -> float:
        """Calculate engagement score"""
        base_engagement = original_message.engagement_score * 0.7
        
        # Questions increase engagement
        if "?" in response:
            base_engagement += 0.15
        
        # Mentions increase engagement
        mention_count = sum(1 for agent in self.agents if agent in response)
        base_engagement += mention_count * 0.05
        
        # Length factor
        word_count = len(response.split())
        if 10 <= word_count <= 30:
            base_engagement += 0.1
        
        return min(base_engagement, 1.0)
    
    def _update_metrics(self, active_agents: set):
        """Update simulation metrics"""
        if not self.messages:
            return
        
        # Calculate metrics
        total_agents = len(self.agents)
        unique_senders = len(set(msg.sender for msg in self.messages))
        reach = unique_senders / total_agents
        
        recent_messages = self.messages[-20:] if len(self.messages) > 20 else self.messages
        avg_engagement = np.mean([msg.engagement_score for msg in recent_messages])
        avg_sentiment = np.mean([msg.sentiment for msg in recent_messages])
        
        # Store metrics
        self.metrics_history["reach"].append(reach)
        self.metrics_history["engagement"].append(avg_engagement)
        self.metrics_history["sentiment"].append(avg_sentiment)
        self.metrics_history["message_count"].append(len(self.messages))
        self.metrics_history["active_agents"].append(len(active_agents))
    
    def _get_recent_public_messages(self, window: int = None) -> List[Message]:
        """Get recent public messages"""
        window = window or Config.MESSAGE_WINDOW
        if not self.messages:
            return []
        return self.messages[-window:]

print("✅ Simulation engine loaded")

## 5. Visualization Methods

Methods for visualizing the simulation results, including network graphs and time-series metrics.

In [ ]:
# Add visualization methods to the simulation class
def visualize_network(self):
    """Visualize the interaction network"""
    plt.figure(figsize=Config.FIGURE_SIZE)
    
    # Calculate node sizes based on influence
    node_sizes = [self.interaction_network.nodes[node].get('influence', 1) * 1000 
                 for node in self.interaction_network.nodes()]
    
    # Color nodes by role
    role_colors = {
        'Policy Analyst': '#1f77b4',
        'Journalist': '#ff7f0e',
        'Economics Professor': '#2ca02c',
        'Business Owner': '#d62728',
        'Social Activist': '#9467bd',
        'Financial Advisor': '#8c564b',
        'International Trade Expert': '#e377c2'
    }
    node_colors = [role_colors.get(self.interaction_network.nodes[node].get('role', ''), '#gray') 
                  for node in self.interaction_network.nodes()]
    
    # Layout
    if Config.NETWORK_LAYOUT == "spring":
        pos = nx.spring_layout(self.interaction_network, k=2, iterations=50)
    else:
        pos = nx.circular_layout(self.interaction_network)
    
    # Draw network
    nx.draw_networkx_nodes(self.interaction_network, pos, 
                          node_color=node_colors, 
                          node_size=node_sizes,
                          alpha=0.8)
    
    nx.draw_networkx_labels(self.interaction_network, pos, 
                           font_size=10, 
                           font_weight='bold')
    
    # Draw edges with weights
    edges = self.interaction_network.edges()
    weights = [self.interaction_network[u][v]['weight'] for u, v in edges]
    nx.draw_networkx_edges(self.interaction_network, pos, 
                          width=[w*0.5 for w in weights],
                          alpha=0.5,
                          edge_color='gray',
                          arrows=True,
                          arrowsize=20)
    
    plt.title("Agent Interaction Network", fontsize=16, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def visualize_metrics(self):
    """Visualize simulation metrics"""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    time_steps = range(len(self.metrics_history["reach"]))
    
    # Reach over time
    axes[0, 0].plot(time_steps, [r*100 for r in self.metrics_history["reach"]], 
                   marker='o', linewidth=2, markersize=6)
    axes[0, 0].set_title("Information Reach Over Time", fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel("Time Step")
    axes[0, 0].set_ylabel("Reach (%)")
    axes[0, 0].grid(True, alpha=0.3)
    
    # Engagement over time
    axes[0, 1].plot(time_steps, self.metrics_history["engagement"], 
                   marker='s', linewidth=2, markersize=6, color='orange')
    axes[0, 1].set_title("Average Engagement Score", fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel("Time Step")
    axes[0, 1].set_ylabel("Engagement Score")
    axes[0, 1].grid(True, alpha=0.3)
    
    # Sentiment evolution
    axes[1, 0].plot(time_steps, self.metrics_history["sentiment"], 
                   marker='^', linewidth=2, markersize=6, color='green')
    axes[1, 0].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    axes[1, 0].set_title("Sentiment Evolution", fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel("Time Step")
    axes[1, 0].set_ylabel("Average Sentiment")
    axes[1, 0].grid(True, alpha=0.3)
    
    # Active agents and messages
    ax2 = axes[1, 1].twinx()
    axes[1, 1].bar(time_steps, self.metrics_history["active_agents"], 
                  alpha=0.5, label="Active Agents")
    ax2.plot(time_steps, self.metrics_history["message_count"], 
            'r-', linewidth=2, label="Total Messages")
    axes[1, 1].set_title("Activity Metrics", fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel("Time Step")
    axes[1, 1].set_ylabel("Active Agents", color='blue')
    ax2.set_ylabel("Total Messages", color='red')
    axes[1, 1].tick_params(axis='y', labelcolor='blue')
    ax2.tick_params(axis='y', labelcolor='red')
    
    plt.tight_layout()
    plt.show()

def generate_report(self) -> pd.DataFrame:
    """Generate detailed simulation report"""
    report_data = []
    
    for msg in self.messages:
        report_data.append({
            'time_step': self.time_step,
            'sender': msg.sender,
            'topic': msg.topic,
            'content': msg.content[:100] + '...' if len(msg.content) > 100 else msg.content,
            'sentiment': round(msg.sentiment, 3),
            'engagement': round(msg.engagement_score, 3),
            'timestamp': msg.timestamp.strftime('%Y-%m-%d %H:%M:%S')
        })
    
    df = pd.DataFrame(report_data)
    
    # Add summary statistics
    summary = {
        'Total Messages': len(self.messages),
        'Unique Participants': len(set(msg.sender for msg in self.messages)),
        'Average Sentiment': round(df['sentiment'].mean(), 3) if not df.empty else 0,
        'Average Engagement': round(df['engagement'].mean(), 3) if not df.empty else 0,
        'Most Active Agent': df['sender'].value_counts().index[0] if not df.empty else 'N/A',
        'Dominant Topic': df['topic'].value_counts().index[0] if not df.empty else 'N/A'
    }
    
    print("\n📊 SIMULATION SUMMARY:")
    print("=" * 50)
    for key, value in summary.items():
        print(f"{key}: {value}")
    
    return df

# Attach methods to the class
InformationDiffusionSimulation.visualize_network = visualize_network
InformationDiffusionSimulation.visualize_metrics = visualize_metrics
InformationDiffusionSimulation.generate_report = generate_report

print("✅ Visualization methods attached")

## 6. Agent Creation

Let's create a diverse set of agents with different roles, personalities, and expertise areas.

In [ ]:
def create_diverse_agents() -> List[Agent]:
    """Create a diverse set of agents for the simulation"""
    agents = [
        Agent(
            name="Dr. Sarah Chen",
            role="Policy Analyst",
            personality="Analytical, data-driven, focuses on evidence and long-term implications",
            knowledge_base=["economic policy", "fiscal responsibility", "data analysis", "public finance"],
            influence_score=1.2
        ),
        Agent(
            name="Yuki Tanaka",
            role="Journalist",
            personality="Curious, investigative, asks challenging questions and seeks transparency",
            knowledge_base=["current events", "public interest", "transparency", "government accountability"],
            influence_score=1.5
        ),
        Agent(
            name="Professor Matsuda",
            role="Economics Professor",
            personality="Educational, provides historical context and theoretical frameworks",
            knowledge_base=["economic theory", "monetary policy", "education", "historical precedents"],
            influence_score=1.3
        ),
        Agent(
            name="Kenji Nakamura",
            role="Business Owner",
            personality="Practical, concerned with real-world impacts on businesses and employment",
            knowledge_base=["business operations", "taxation", "employment", "market dynamics"],
            influence_score=1.0
        ),
        Agent(
            name="Aiko Suzuki",
            role="Social Activist",
            personality="Passionate about social justice, advocates for marginalized communities",
            knowledge_base=["social welfare", "inequality", "public services", "community needs"],
            influence_score=1.1
        ),
        Agent(
            name="Hiroshi Yamamoto",
            role="Financial Advisor",
            personality="Risk-aware, focuses on investment implications and market stability",
            knowledge_base=["investment", "risk management", "market analysis", "portfolio theory"],
            influence_score=0.9
        ),
        Agent(
            name="Mei Wong",
            role="International Trade Expert",
            personality="Global perspective, analyzes international implications and comparisons",
            knowledge_base=["international trade", "global economics", "trade policy", "currency markets"],
            influence_score=1.1
        )
    ]
    
    # Establish initial connections
    for i, agent in enumerate(agents):
        # Each agent knows 2-4 other agents initially
        num_connections = random.randint(2, 4)
        possible_connections = [a.name for j, a in enumerate(agents) if i != j]
        agent.connections = random.sample(possible_connections, min(num_connections, len(possible_connections)))
    
    return agents

# Create and display agents
agents = create_diverse_agents()
print(f"Created {len(agents)} agents:\n")
for agent in agents:
    print(f"👤 {agent.name} - {agent.role}")
    print(f"   Expertise: {', '.join(agent.knowledge_base[:2])}...")
    print(f"   Connections: {', '.join(agent.connections[:3])}")
    print()

## 7. Running the Simulation

Now let's run a complete simulation and observe how information diffuses through our agent network.

In [ ]:
def run_full_simulation():
    """Run a complete simulation with visualization"""
    
    print("🚀 INITIALIZING LLM AGENT SIMULATION")
    print("=" * 60)
    
    # Create agents
    agents = create_diverse_agents()
    print(f"✅ Created {len(agents)} diverse agents")
    
    # Define topics
    topics = [
        "Ministry of Finance fiscal policy announcement",
        "Economic stimulus measures",
        "Tax reform proposals",
        "Public spending priorities"
    ]
    
    # Initialize simulation
    sim = InformationDiffusionSimulation(agents, topics)
    
    # Seed information with high urgency
    print("\n🌱 SEEDING INITIAL INFORMATION")
    sim.seed_information(
        topic=topics[0],
        seed_agents=["Dr. Sarah Chen", "Yuki Tanaka"],
        urgency="high"
    )
    
    print("\n⏱️  RUNNING SIMULATION")
    print("=" * 60)
    
    # Run simulation steps
    for step in range(10):
        print(f"\n--- Time Step {step + 1} ---")
        sim.simulate_time_step()
        
        # Add secondary information injection
        if step == 4:
            print("\n🌱 INJECTING SECONDARY INFORMATION")
            sim.seed_information(
                topic=topics[1],
                seed_agents=["Professor Matsuda"],
                urgency="normal"
            )
    
    # Generate report
    print("\n📊 GENERATING REPORT")
    report_df = sim.generate_report()
    
    # Visualizations
    print("\n📈 CREATING VISUALIZATIONS")
    sim.visualize_metrics()
    sim.visualize_network()
    
    return sim, report_df

# Run the simulation
# Note: Uncomment the line below to run the simulation with actual API calls
# simulation, report = run_full_simulation()

## 8. Demonstration Mode

For demonstration purposes without API calls, let's create a mock simulation that shows the structure.

In [ ]:
# Demo simulation without API calls
def run_demo_simulation():
    """Run a demo simulation without actual API calls"""
    print("🎭 RUNNING DEMO SIMULATION (No API calls)")
    print("=" * 60)
    
    # Create agents
    agents = create_diverse_agents()
    
    # Initialize simulation
    topics = ["Fiscal Policy Demo", "Economic Measures Demo"]
    sim = InformationDiffusionSimulation(agents, topics)
    
    # Manually add some demo messages
    demo_messages = [
        Message(
            sender="Dr. Sarah Chen",
            content="The new fiscal policy requires careful analysis of long-term impacts.",
            timestamp=datetime.now(),
            topic=topics[0],
            sentiment=0.3,
            engagement_score=0.8
        ),
        Message(
            sender="Yuki Tanaka",
            content="What are the transparency measures in this new policy?",
            timestamp=datetime.now(),
            topic=topics[0],
            sentiment=0.1,
            engagement_score=0.9
        ),
        Message(
            sender="Professor Matsuda",
            content="Historical precedents suggest we should consider inflation risks.",
            timestamp=datetime.now(),
            topic=topics[0],
            sentiment=-0.2,
            engagement_score=0.7
        )
    ]
    
    # Add messages to simulation
    sim.messages.extend(demo_messages)
    
    # Update metrics manually
    for i in range(5):
        sim.metrics_history["reach"].append(0.2 + i * 0.1)
        sim.metrics_history["engagement"].append(0.6 + random.uniform(-0.1, 0.1))
        sim.metrics_history["sentiment"].append(0.1 + random.uniform(-0.2, 0.2))
        sim.metrics_history["message_count"].append(len(demo_messages) + i * 2)
        sim.metrics_history["active_agents"].append(3 + random.randint(0, 2))
    
    # Add network edges
    sim._update_network("Dr. Sarah Chen", "Yuki Tanaka", demo_messages[0])
    sim._update_network("Yuki Tanaka", "Professor Matsuda", demo_messages[1])
    sim._update_network("Professor Matsuda", "Dr. Sarah Chen", demo_messages[2])
    
    # Visualize
    print("\n📊 Demo Metrics:")
    sim.visualize_metrics()
    
    print("\n🌐 Demo Network:")
    sim.visualize_network()
    
    return sim

# Run demo
demo_sim = run_demo_simulation()

## 9. Twitter Data Analysis

Functions for loading and analyzing the Twitter data to extract topics and high-engagement content.

In [ ]:
def analyze_twitter_data(csv_path: str):
    """Load and analyze the Twitter data for integration"""
    try:
        # Try different encodings
        for encoding in ['utf-16', 'utf-16-le', 'utf-8', 'iso-8859-1']:
            try:
                df = pd.read_csv(csv_path, encoding=encoding, sep='\t')
                print(f"✅ Successfully loaded data with {encoding} encoding")
                break
            except:
                continue
        
        print(f"📊 Data shape: {df.shape}")
        print(f"🔤 Columns: {list(df.columns)[:5]}...")  # Show first 5 columns
        
        # Extract topics and high-engagement content
        if 'Likes' in df.columns or any('like' in col.lower() for col in df.columns):
            # Find high-engagement posts
            high_engagement = df.nlargest(10, 'Likes') if 'Likes' in df.columns else df.head(10)
            
            print("\n🔥 High Engagement Topics:")
            for idx, row in high_engagement.iterrows():
                content = str(row.get('Content', row.get('Text', 'N/A')))[:100]
                likes = row.get('Likes', 'N/A')
                print(f"- {content}... (Likes: {likes})")
        
        return df
        
    except Exception as e:
        print(f"❌ Error loading Twitter data: {e}")
        return None

# Example usage (uncomment with your file path)
# twitter_df = analyze_twitter_data("財務省_AND_likes130_2025_02052025_0522.csv")
print("Twitter data analysis function ready")

## 10. Advanced Analysis Tools

Additional tools for analyzing simulation results and extracting insights.

In [ ]:
def analyze_influence_patterns(sim: InformationDiffusionSimulation):
    """Analyze influence patterns in the simulation"""
    if not sim.messages:
        print("No messages to analyze")
        return
    
    # Create influence matrix
    agents = list(sim.agents.keys())
    influence_matrix = pd.DataFrame(0, index=agents, columns=agents)
    
    # Count interactions
    for edge in sim.interaction_network.edges():
        sender, receiver = edge
        weight = sim.interaction_network[sender][receiver]['weight']
        influence_matrix.loc[sender, receiver] = weight
    
    # Visualize influence matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(influence_matrix, annot=True, fmt='d', cmap='Blues', 
                cbar_kws={'label': 'Number of Interactions'})
    plt.title('Agent Influence Matrix', fontsize=16, fontweight='bold')
    plt.xlabel('Influenced Agent')
    plt.ylabel('Influencing Agent')
    plt.tight_layout()
    plt.show()
    
    # Calculate influence scores
    influence_scores = influence_matrix.sum(axis=1).sort_values(ascending=False)
    print("\n📊 Top Influencers:")
    for agent, score in influence_scores.head().items():
        print(f"  {agent}: {score} influence points")

def analyze_topic_evolution(sim: InformationDiffusionSimulation):
    """Analyze how topics evolve over time"""
    if not sim.messages:
        print("No messages to analyze")
        return
    
    # Group messages by time windows
    time_windows = 5
    messages_per_window = len(sim.messages) // time_windows
    
    topic_evolution = []
    
    for i in range(time_windows):
        start = i * messages_per_window
        end = (i + 1) * messages_per_window if i < time_windows - 1 else len(sim.messages)
        
        window_messages = sim.messages[start:end]
        topic_counts = {}
        
        for msg in window_messages:
            topic_counts[msg.topic] = topic_counts.get(msg.topic, 0) + 1
        
        topic_evolution.append(topic_counts)
    
    # Visualize topic evolution
    topics = list(set(msg.topic for msg in sim.messages))
    plt.figure(figsize=(12, 6))
    
    for topic in topics:
        counts = [window.get(topic, 0) for window in topic_evolution]
        plt.plot(range(1, time_windows + 1), counts, marker='o', label=topic, linewidth=2)
    
    plt.xlabel('Time Window')
    plt.ylabel('Number of Messages')
    plt.title('Topic Evolution Over Time', fontsize=16, fontweight='bold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

print("Advanced analysis tools loaded")

## 11. Conclusions and Next Steps

### Summary

This notebook implements a sophisticated information diffusion simulation using real LLM agents. Key achievements:

1. **Real LLM Integration**: Replaced pseudo-random generation with actual API calls to OpenAI or Anthropic
2. **Network Dynamics**: Agents form connections and influence each other based on interactions
3. **Advanced Metrics**: Comprehensive tracking of sentiment, engagement, and network effects
4. **Rich Visualizations**: Network graphs, time-series analysis, and influence matrices
5. **Extensibility**: Framework ready for Twitter data integration and custom scenarios

### Next Steps

To enhance this simulation further:

1. **Fine-tune Agent Behaviors**
   - Calibrate response probabilities based on real social media data
   - Implement more sophisticated personality models
   - Add emotional states and opinion dynamics

2. **Enhance Information Diffusion Models**
   - Implement epidemic-style diffusion models (SIR, SEIR)
   - Add network effects like homophily and preferential attachment
   - Model information decay and reinforcement

3. **Twitter Data Integration**
   - Extract real topics and sentiment patterns
   - Create agent personalities based on user archetypes
   - Seed simulations with actual trending discussions

4. **Performance Optimization**
   - Implement asynchronous API calls
   - Add response caching
   - Batch similar queries

5. **Advanced Analytics**
   - Topic modeling with LDA or BERT
   - Community detection algorithms
   - Influence propagation analysis

### Usage Notes

- Always set your API keys as environment variables
- Monitor API usage to control costs
- Use demo mode for testing without API calls
- Adjust Config parameters based on your needs

This simulation provides a powerful framework for studying information diffusion in AI-mediated communication networks.